In [1]:
from google.colab import drive
drive.mount('/content/drive')
COLAB_PATH = "/content/drive/MyDrive/Colab Notebooks"
REPO_PATH = "/content/drive/MyDrive/style-transfer-paraphrase"

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Execute the readme setup from the [repo](https://github.com/martiansideofthemoon/style-transfer-paraphrase)

In [2]:
# %cd "/content/drive/MyDrive"
# !git clone https://github.com/ktxlh/style-transfer-paraphrase.git

%cd $REPO_PATH

!pip install torch torchvision
!pip install -r requirements.txt
!pip install --editable .

!cd fairseq
!pip install --editable .

!pip install fairseq bitarray # workaround: to resolve some Colab specific module import error (https://github.com/pytorch/fairseq/issues/3093)

/content/drive/MyDrive/style-transfer-paraphrase
  Using cached en_core_web_sm-2.3.1-py3-none-any.whl
Obtaining file:///content/drive/My%20Drive/style-transfer-paraphrase
  Attempting uninstall: style-paraphrase
    Found existing installation: style-paraphrase 1.0
    Can't uninstall 'style-paraphrase'. No files were found to uninstall.
  Running setup.py develop for style-paraphrase
Obtaining file:///content/drive/My%20Drive/style-transfer-paraphrase
  Attempting uninstall: style-paraphrase
    Found existing installation: style-paraphrase 1.0
    Can't uninstall 'style-paraphrase'. No files were found to uninstall.
  Running setup.py develop for style-paraphrase


**`Restart runtime` now for the first time you run this notebook in each session**

In [3]:
%cd /content/
!wget https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
!tar -xzvf roberta.large.tar.gz

# Add the following to your .bashrc file, feel free to store the model elsewhere on the hard disk
!export ROBERTA_LARGE=$PWD/roberta.large
%cd $REPO_PATH

/content
--2021-12-04 02:28:14--  https://dl.fbaipublicfiles.com/fairseq/models/roberta.large.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.75.142, 104.22.74.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 655283069 (625M) [application/gzip]
Saving to: ‘roberta.large.tar.gz’

roberta.large.tar.g 100%[===================>] 624.93M  12.0MB/s    in 54s     

2021-12-04 02:29:09 (11.6 MB/s) - ‘roberta.large.tar.gz’ saved [655283069/655283069]

roberta.large/
roberta.large/dict.txt
roberta.large/model.pt
roberta.large/NOTE
/content/drive/MyDrive/style-transfer-paraphrase


custom data

In [4]:
aave_data_name, sae_data_name = "aae", "tweets"
# !mkdir datasets/$aave_data_name datasets/$sae_data_name

In [5]:
import os
import pandas as pd
data_input_path = f"{COLAB_PATH}/AAVE_SAE/data/training"
split_paths = {x.split('_')[0]: os.path.join(data_input_path, x) for x in os.listdir(data_input_path)}

split_name_map = {
    'train': 'train',
    'val': 'dev',
    'test': 'test',
}
def conver_to_strap_data(split_name, split_path):
    df = pd.read_csv(split_path)
    aave_x = df.sentence[df.label==1].tolist()
    sae_x = df.sentence[df.label==0].tolist()
    with open(os.path.join('datasets', aave_data_name, split_name_map[split_name] + '.txt'), 'w') as f:
        f.write('\n'.join(aave_x) + '\n')
    with open(os.path.join('datasets', sae_data_name, split_name_map[split_name] + '.txt'), 'w') as f:
        f.write('\n'.join(sae_x) + '\n')
    with open(os.path.join('datasets', aave_data_name, split_name_map[split_name] + '.label'), 'w') as f:
        f.write(''.join([aave_data_name + '\n' for _ in range(len(aave_x))]))
    with open(os.path.join('datasets', sae_data_name, split_name_map[split_name] + '.label'), 'w') as f:
        f.write(''.join([sae_data_name + '\n' for _ in range(len(sae_x))]))

for k, v in split_paths.items():
    conver_to_strap_data(k, v)

BPE conversion: steps from readme.md

1. To convert a plaintext dataset into it's BPE form run the command,

In [7]:
!python datasets/dataset2bpe.py --dataset datasets/$aave_data_name
!python datasets/dataset2bpe.py --dataset datasets/$sae_data_name

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
fatal: not a git repository (or any of the parent directories): .git
running build_ext
/usr/local/lib/python3.7/dist-packages/torch/utils/cpp_extension.py:381: UserWarning: Attempted to use ninja as the BuildExtension backend but we could not find ninja.. Falling back to using the slow distutils backend.
  warnings.warn(msg.format('we could not find ninja.'))
skipping 'fairseq/data/data_utils_fast.cpp' Cython extension (up-to-date)
skipping 'fairseq/data/token_block_utils_fast.cpp' Cython extension (up-to-date)
building 'alignment_train_cpu_binding' extension
x86_64-linux-gnu-gcc -pthread -Wno-unused-result -Wsign-compare -DNDEBUG -g -fwrapv -O2 -Wall -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -g -fdebug-prefix-map=/build/python3.7-Y7dWVB/python3.7-3.7.12=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE

2. Next, for converting the BPE codes to fairseq binaries and building a label dictionary, first make sure you have downloaded RoBERTa and setup the $ROBERTA_LARGE global variable in your .bashrc (see "Setup" for more details). Then run,

In [8]:
!chmod +x datasets/bpe2binary.sh
!cp "/content/roberta.large/dict.txt" /dict.txt
# !datasets/bpe2binary.sh datasets/$aave_data_name
!datasets/bpe2binary.sh datasets/$sae_data_name

2021-12-04 02:32:43 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, bf16=False, bpe=None, checkpoint_shard_count=1, checkpoint_suffix='', cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='datasets/tweets-bin/label', empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, only_source=True, optimizer=None, padding_factor=8, profile=False, quantization_config_path=None, scoring='bleu', seed=1, source_lang=None, srcdict=None, target_lang=None, task='translation', tensorboard_logdir=None, testpref=None, tgtdict=None, threshold_loss_scale=None, thresholdsrc=0, thresholdtgt=0, tokenizer=None, tpu=False, trainp

3. To train inverse paraphrasers you will need to paraphrase the dataset. First, download the pretrained model paraphraser_gpt2_large from here. After downloading the pretrained paraphrase model run the command,

In [9]:
# !python datasets/paraphrase_splits.py --dataset datasets/$aave_data_name
!python datasets/paraphrase_splits.py --dataset datasets/$sae_data_name

Using cache found in /root/.cache/torch/hub/pytorch_fairseq_main
2021-12-04 02:33:03 | INFO | fairseq.file_utils | loading archive file http://dl.fbaipublicfiles.com/fairseq/models/roberta.base.tar.gz from cache at /root/.cache/torch/pytorch_fairseq/37d2bc14cf6332d61ed5abeb579948e6054e46cc724c7d23426382d11a31b2d6.ae5852b4abc6bf762e0b6b30f19e741aa05562471e9eb8f4a6ae261f04f9b350
/usr/local/lib/python3.7/dist-packages/hydra/experimental/initialize.py:36: UserWarning: hydra.experimental.initialize() is no longer experimental. Use hydra.initialize()
  message="hydra.experimental.initialize() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/experimental/compose.py:19: UserWarning: hydra.experimental.compose() is no longer experimental. Use hydra.compose()
  message="hydra.experimental.compose() is no longer experimental."
/usr/local/lib/python3.7/dist-packages/hydra/core/default_element.py:126: UserWarning: In 'config': Usage of deprecated keyword in package header '#

Step 4 and 5 are for training. We don't need them.
> 4. Add an entry to the DATASET_CONFIG dictionary in style_paraphrase/dataset_config.py, customizing configuration if needed.
> 5. Enter your dataset in the hyperparameters file (style_paraphrase/hyperparameters_config.py) and run python style_paraphrase/schedule.py.


In [17]:
!chmod +x "$REPO_PATH/style_paraphrase/evaluation/scripts/evaluate_twitter.sh"
!style_paraphrase/evaluation/scripts/evaluate_twitter.sh \
    "$REPO_PATH/aae" "$REPO_PATH/tweets" "$REPO_PATH/paraphraser_gpt2_large"


-------------------------------------------
Mode nucleus_paraphrase  --- top-p 0.0, split test
-------------------------------------------


translate tweets to aae
Output already exists...

translate aae to tweets
Output already exists...

RoBERTa test classification

tcmalloc: large alloc 1425891328 bytes == 0x55ea8ea8e000 @  0x7fa335aa7b6b 0x7fa335ac7379 0x7fa21f2bdcde 0x7fa21f2bf452 0x7fa271d31749 0x7fa31ab5ad59 0x55ea5e277045 0x55ea5e237c52 0x55ea5e2aac25 0x55ea5e2a5ced 0x55ea5e2387f3 0x55ea5e2382f9 0x55ea5e37f35d 0x55ea5e2eea0b 0x55ea5e2373a1 0x55ea5e328e1d 0x55ea5e2aae99 0x55ea5e2a5ced 0x55ea5e177e2b 0x55ea5e2a7fe4 0x55ea5e2a59ee 0x55ea5e238bda 0x55ea5e2a7737 0x55ea5e2a59ee 0x55ea5e238bda 0x55ea5e2a6915 0x55ea5e2a59ee 0x55ea5e238bda 0x55ea5e2a7737 0x55ea5e2a5ced 0x55ea5e177e2b
tcmalloc: large alloc 1425891328 bytes == 0x55eae3a64000 @  0x7fa335aa7b6b 0x7fa335ac7379 0x7fa21f2bdcde 0x7fa21f2bf452 0x7fa271d31749 0x7fa31ab5ad59 0x55ea5e277045 0x55ea5e237c52 0x55ea5e2aac25 0x55ea5e2

We also convert the BPE-ed AAVE and SAE input of STRAP to text for our baseline comparison.

In [10]:
# from transformers import GPT2Tokenizer
# if not os.path.isdir(os.path.join(f'{REPO_PATH}', 'converted_texts')):
#     os.mkdir(os.path.join(f'{REPO_PATH}', 'converted_texts'))
# for s in ['aae', 'tweets']:
#     if not os.path.isdir(os.path.join(f'{REPO_PATH}', 'converted_texts', s)):
#         os.mkdir(os.path.join(f'{REPO_PATH}', 'converted_texts', s))

#     tokenizer = GPT2Tokenizer.from_pretrained(f"{REPO_PATH}/{s}")
#     bpe_fnames = list(filter(lambda x: x.split('.')[-1] == 'bpe', os.listdir(f"{REPO_PATH}/cds/{s}")))
#     for fname in bpe_fnames:
#         lines = list(map(lambda x: list(map(lambda y: int(y), x.split())), open(f"{REPO_PATH}/cds/{s}/{fname}").readlines()))
#         texts = list(map(lambda line: tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(line)), lines))
#         with open(f"/content/drive/MyDrive/converted_texts/{s}/{fname.replace('bpe', 'txt')}", 'w') as f:
#             f.write('\n'.join(texts) + '\n')